In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import transforms
from tqdm import tqdm
import warnings
warnings.filterwarnings(action="ignore")

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

# defining our training and test sets
train_dataset = FashionMNIST(root=".", train=True, download=True, transform=transform)
test_dataset = FashionMNIST(root=".", train=False, download=True, transform=transform)

# creating our dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=4*4*64, out_features=512)
        self.drop1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.drop2 = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(in_features=256, out_features=128)
        self.drop3 = nn.Dropout(p=0.3)
        self.fc4 = nn.Linear(in_features=128, out_features=64)
        self.drop4 = nn.Dropout(p=0.3)
        self.fc5 = nn.Linear(in_features=64, out_features=10)
      
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = F.relu(self.fc3(x))
        x = self.drop3(x)
        x = F.relu(self.fc4(x))
        x = self.drop4(x)
        x = F.relu(self.fc5(x))
        x = F.log_softmax(x, dim=1)
        return x

# Creating model
model = Classifier()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0

    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Compute average training loss
    train_loss /= len(train_loader)

    # Evaluation
    model.eval()
    test_loss = 0.0
    test_correct = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()

    # Compute average test loss
    test_loss /= len(test_loader)

    # Compute test accuracy
    test_accuracy = 100.0 - test_loss

    # Print training and testing statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, "
          f"Test Accuracy: {test_accuracy:.2f}%")

100%|██████████| 938/938 [00:27<00:00, 34.04it/s]


Epoch [1/10], Train Loss: 0.7541, Test Accuracy: 99.56%


100%|██████████| 938/938 [00:27<00:00, 34.15it/s]


Epoch [2/10], Train Loss: 0.4044, Test Accuracy: 99.66%


100%|██████████| 938/938 [00:27<00:00, 33.93it/s]


...

Epoch [8/10], Train Loss: 0.2198, Test Accuracy: 99.72%


100%|██████████| 938/938 [00:27<00:00, 33.83it/s]


Epoch [9/10], Train Loss: 0.2068, Test Accuracy: 99.73%


100%|██████████| 938/938 [00:27<00:00, 33.91it/s]


Epoch [10/10], Train Loss: 0.1978, Test Accuracy: 99.72%
